In [56]:
###DEBUG###
import os

os.environ['API_KEY'] = "889baad2-02b0-4c78-abe1-1952700d33ef" 
os.environ['NONINTERACT'] = 'True'

In [57]:
import json
import math
import os
import random
import sys
import time
import gc

In [58]:
import xarray as xr
import numpy as np
import pandas as pd
import random

import qnt.ta as qnta
import qnt.data as qndata
import qnt.backtester as qnbt

import qnt.stats as qnstats
import qnt.output as output
import qnt.graph as qngraph

In [59]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [60]:
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

class ElecDataset(Dataset):
    def __init__(self,feature,target):
        self.feature = feature
        self.target = target
    
    def __len__(self):
        return len(self.feature)
    
    def __getitem__(self,idx):
        item = self.feature[idx]
        label = self.target[idx]
        
        return item, label

In [61]:
def load_data():
    data = qndata.cryptofutures.load_data(#min_date = "2002-01-01", 
                                    max_date = "2013-12-31", 
                                    #tail=period,
                                   )
    return data

data = load_data()
close = data.sel(field="close").ffill('time').bfill('time')#.fillna(0)
asset_name_all = data.coords['asset'].values
print(list(asset_name_all))
print(len(asset_name_all))

| |#                                               | 6896 Elapsed Time: 0:00:00


['BTC']
1


In [62]:
in_seq1 = []

for asset in asset_name_all:
    single_close = close.sel(asset=[asset]).to_pandas()
    print(single_close)
    close_ret = np.log(single_close/single_close.shift(1)).dropna()
    close_arr = np.array(close_ret)
    in_seq1.extend(close_arr)

asset            BTC
time                
2013-04-01  104.0100
2013-04-02  115.7625
2013-04-03  123.0000
2013-04-04  134.1200
2013-04-05  144.5000
...              ...
2013-12-27  711.0400
2013-12-28  721.1193
2013-12-29  728.8000
2013-12-30  736.4402
2013-12-31  728.0000

[275 rows x 1 columns]


In [63]:
#scaling 
from sklearn.preprocessing import MinMaxScaler
scaler_close = MinMaxScaler()
close_scaled = scaler_close.fit_transform(in_seq1)

In [64]:
# choose a number of time steps
n_steps = 50
# split into samples
X, y = split_sequence(close_scaled, n_steps)
y_reverse = scaler_close.inverse_transform(y)
y = np.where(y_reverse < 0, -1, 1 )

In [74]:
split = int(len(X)*0.8)
train_x = np.array(X)
valid_x = np.array(X[split:])
train_y = y
valid_y = y[split:]

train = ElecDataset(train_x.reshape(train_x.shape[0], 1, train_x.shape[1]),train_y)
valid = ElecDataset(valid_x.reshape(valid_x.shape[0], 1, valid_x.shape[1]),valid_y)
train_loader = torch.utils.data.DataLoader(train,batch_size=64,shuffle=True, drop_last = True)
print(len(train_loader))
valid_loader = torch.utils.data.DataLoader(valid,batch_size=64,shuffle=True, drop_last = True)
print(len(valid_loader))

3
0


CNN-1D Model

In [66]:
class TS(nn.Module):
    def __init__(self, n_input= 1, n_output=1):
        super(TS, self).__init__()

        self.conv1 = nn.Conv1d(1, 1, kernel_size = 3)
        self.conv2 = nn.Conv1d(1, 1, kernel_size = 3)
        self.pool1d = nn.MaxPool1d(2,2)
        self.flatten = nn.Flatten()
        self.relu = nn.ReLU(inplace=True)
        self.fc1 = nn.Linear(11,4)
        self.fc2 = nn.Linear(4, n_output)
        #self.double()

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool1d(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool1d(x)
        # x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


model = TS(n_input=1, n_output=1)
model.to(device)
print(model)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

n = count_parameters(model)
print("Number of parameters: %s" % n)

TS(
  (conv1): Conv1d(1, 1, kernel_size=(3,), stride=(1,))
  (conv2): Conv1d(1, 1, kernel_size=(3,), stride=(1,))
  (pool1d): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (relu): ReLU(inplace=True)
  (fc1): Linear(in_features=11, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=1, bias=True)
)
Number of parameters: 61


In [67]:
train_losses = []
valid_losses = []

def Train():
    
    running_loss = .0
    
    model.train()
    
    for idx, (inputs, labels) in enumerate(train_loader):
        labels = labels.unsqueeze(1)
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        preds = model(inputs.float())
        loss = criterion(preds,labels.float())
        loss.backward()
        optimizer.step()
        running_loss += loss
        
    train_loss = running_loss/len(train_loader)
    train_losses.append(train_loss.cpu().detach().numpy())
    
    print(f'train_loss {train_loss}')

def Valid():
    running_loss = .0
    
    model.eval()
    
    with torch.no_grad():
        for idx, (inputs, labels) in enumerate(valid_loader):
            labels = labels.unsqueeze(1)
            inputs = inputs.to(device)
            #print(inputs)
            labels = labels.to(device)
            #print(labels)
            optimizer.zero_grad()
            preds = model(inputs.float())
            #print(preds)
            loss = criterion(preds,labels)
            running_loss += loss
            
        valid_loss = running_loss/len(valid_loader)
        valid_losses.append(valid_loss.cpu().detach().numpy())
        print(f'valid_loss {valid_loss}')

In [83]:
epochs = 5
for epoch in range(epochs):
    print('epochs {}/{}'.format(epoch+1,epochs))
    Train()
    #Valid()
    gc.collect()

epochs 1/5
train_loss 0.9769309759140015
epochs 2/5
train_loss 0.975398063659668
epochs 3/5
train_loss 0.9653345942497253
epochs 4/5
train_loss 0.974345862865448
epochs 5/5
train_loss 0.9654219150543213


In [84]:
def save_ckp(state, checkpoint_dir):
        f_path = checkpoint_dir
        torch.save(state, f_path)

checkpoint = {
                'epoch': epoch + 1,
                'state_dict': model.state_dict(),
                'optimizer': optimizer.state_dict()
            }
save_path = 'generalcheckpoint.pt'
save_ckp(checkpoint, save_path)

In [85]:
# https://quantiacs.com/documentation/en/examples/machine_learning_ridgeclassifier.html

import xarray as xr

import qnt.backtester as qnbt
import qnt.data as qndata
import numpy as np
import logging
import torch

def load_data(period):
    return qndata.cryptofutures.load_data(
        tail=period)

def predict_weights(market_data):
    
    def save_ckp(state, checkpoint_dir):
        f_path = checkpoint_dir
        torch.save(state, f_path)
  
    def load_ckp(checkpoint_fpath, model, optimizer):
        checkpoint = torch.load(checkpoint_fpath)
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        return model, optimizer, checkpoint['epoch']
        
    def define_model():
        model = TS(n_input=1, n_output=1)
        return model

    def Train():
        running_loss = .0
        model.train()
        
        for idx, (inputs, labels) in enumerate(train_loader):
            labels = labels.unsqueeze(1)
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            preds = model(inputs.float())
            loss = criterion(preds,labels.float())
            loss.backward()
            optimizer.step()
            running_loss += loss
            
        train_loss = running_loss/len(train_loader)
        train_losses.append(train_loss.cpu().detach().numpy())
        #print(f'train_loss {train_loss}')

    data = market_data.copy(True)
    close = data.sel(field="close").ffill('time').bfill('time').fillna(0).to_pandas()
    asset_name_all = data.coords['asset'].values
    predict_weights_next_day_df = data.sel(field="close").isel(time=-1).to_pandas()

    for asset_name in asset_name_all:
        close_returns = np.log(close[asset_name]/close[asset_name].shift(1)).dropna()
        close_array = np.array(close_returns)
        in_seq1 = close_array[:-1]
        
        #scaling
        close_scaled = scaler_close.transform(in_seq1.reshape(-1, 1))
        # choose a number of time steps
        n_steps = 50
        # split into samples
        X, y = split_sequence(close_scaled, n_steps)
        y_reverse = scaler_close.inverse_transform(y)
        y = np.where(y_reverse < 0, -1, 1 )

        # dataloader
        train = ElecDataset(X.reshape(X.shape[0],1, X.shape[1])[-5:],y[-5:])
        train_loader = torch.utils.data.DataLoader(train,batch_size=1,shuffle=True)
        
        # model and optimizer
        model = define_model()
        model.to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
        criterion = nn.MSELoss()

        try:
            # load from 
            try:
                ckp_path = asset_name+'checkpoint.pt'
                model, optimizer, start_epoch = load_ckp(ckp_path, model, optimizer)
                #print("loading asset model")
            except: 
                ckp_path = 'generalcheckpoint.pt'
                model, optimizer, start_epoch = load_ckp(ckp_path, model, optimizer)
                #print("loading general model") 

            # train the model with current data
            epochs = 1
            train_losses = []
            for epoch in range(epochs):
                #print('epochs {}/{}'.format(epoch+1,epochs))
                Train()
                gc.collect()

            # save the model
            checkpoint = {
                'epoch': epoch + 1,
                'state_dict': model.state_dict(),
                'optimizer': optimizer.state_dict()
            }
            save_path = asset_name+'checkpoint.pt'
            save_ckp(checkpoint, save_path)
            
            x = close_scaled[-n_steps:]
            inputs = torch.from_numpy(x.reshape(1, 1, n_steps))
            
            model.eval()
            with torch.no_grad():
                inputs = inputs.to(device)
                optimizer.zero_grad()
                preds = model(inputs.float())
           
            predict_weights_next_day_df[asset_name] = preds[0]

        except:
            logging.exception("model failed")
            # if there is exception, return zero values
            return xr.zeros_like(data.isel(field=0, time=0))

    #print(predict_weights_next_day_df)
    return predict_weights_next_day_df.to_xarray()


weights = qnbt.backtest(
    competition_type="cryptofutures",
    load_data=load_data,
    lookback_period=100,
    start_date='2014-01-01',
    #test_period = 3,
    strategy=predict_weights,
    analyze=True,
    build_plots=True
)

Run last pass...
Load data...
Run strategy...
Load data for cleanup...
Output cleaning...
fix uniq
Normalization...
Output cleaning is complete.
Write result...


NOTICE: The environment variable OUTPUT_PATH was not specified. The default value is 'fractions.nc.gz'


Write output: fractions.nc.gz
---
Run first pass...
Load data...


| |#                                               | 2898 Elapsed Time: 0:00:00


Run strategy...
---
Load full data...


...

---
Align...
Calc global stats...
---
Calc stats per asset...
Build plots...
---
Output:


asset,BTC
time,
2021-07-14,-0.037845
2021-07-15,-0.039965
2021-07-16,-0.041220
2021-07-17,-0.042342
2021-07-18,-0.044137
2021-07-19,-0.046359
2021-07-20,-0.049206
2021-07-21,-0.053162
2021-07-22,-0.058415


Stats:


field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2021-07-14,2.527495,-0.000331,0.115302,-0.050222,-0.138324,1.134705,0.130833,-1.0,1.0,0.005430,54.819397
2021-07-15,2.530505,0.001191,0.115281,-0.049091,-0.138324,1.136017,0.130961,-1.0,1.0,0.005428,54.819397
2021-07-16,2.530248,-0.000102,0.115260,-0.049188,-0.138324,1.135651,0.130895,-1.0,1.0,0.005428,54.819397
2021-07-17,2.530628,0.000150,0.115239,-0.049045,-0.138324,1.135613,0.130867,-1.0,1.0,0.005426,54.819397
2021-07-18,2.530894,0.000105,0.115218,-0.048945,-0.138324,1.135517,0.130832,-1.0,1.0,0.005424,54.819397
2021-07-19,2.533761,0.001133,0.115198,-0.047867,-0.138324,1.136752,0.130951,-1.0,1.0,0.005423,54.819397
2021-07-20,2.537963,0.001658,0.115178,-0.046288,-0.138324,1.138663,0.131149,-1.0,1.0,0.005422,54.819397
2021-07-21,2.529360,-0.003390,0.115165,-0.049521,-0.138324,1.133941,0.130590,-1.0,1.0,0.005422,54.819397
2021-07-22,2.527297,-0.000816,0.115145,-0.050297,-0.138324,1.132642,0.130418,-1.0,1.0,0.005420,54.819397


---


1. 2020, input_dim = 50, 1 epoch
sr = 1.47
2. 2013, input_dim = 50, 1 epoch
sr = 0.587
3. 2019, input_dim = 50, 1 epoch
sr = 1.12 
4. 2019, input dim = 50, 2 epochs
sr = 1.32
5. 2019, input dim = 40, 2 epochs
sr = 1.49
6. 2019, input dim = 30, 2 epochs
sr = 1.33
7. 2017, input dim = 40, 2 epochs
sr = 0.61
8. 2017, input dim = 40, 1 epochs
sr = 0.59
9. 2019, input dim = 40, 1 epochs
sr = 1.12
10. 2006, input dim = 40, 1 epoch (13 assets)
sr = 0.55
10. 2006, input dim = 60, 1 epoch (10 assets)
sr = 0.53